<a href="https://colab.research.google.com/github/aaas24/colabs/blob/main/cs68_hw0_starter_aaas24ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from vega_datasets import data
import seaborn as sns


In [74]:
import pandas_profiling
#pd.set_option('max_rows', 99999)
pd.set_option('max_rows', 500)

In [75]:
housing_data =  pd.read_csv('https://raw.githubusercontent.com/cjflanagan/cs68/master/housing.csv')

## Code provided by Prof

In [76]:
# Data description available here: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=data_description.txt
housing_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [77]:
housing_data.shape

(1460, 81)

In [78]:
sns.scatterplot(y='SalePrice', x='LotArea', data=housing_data)

In [79]:
sns.lmplot(y='SalePrice', x='LotArea', data=housing_data)

In [80]:
y = housing_data['SalePrice']
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [81]:
X = housing_data[['LotArea', 'BedroomAbvGr', 'OverallQual']]
X.head()

,LotArea,BedroomAbvGr,OverallQual
0,8450,3,7
1,9600,3,6
2,11250,3,7
3,9550,3,7
4,14260,4,8


In [82]:
X = sm.add_constant(X)
X.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,const,LotArea,BedroomAbvGr,OverallQual
0,1.0,8450,3,7
1,1.0,9600,3,6
2,1.0,11250,3,7
3,1.0,9550,3,7
4,1.0,14260,4,8


## Model to Beat

In [83]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     955.7
Date:                Sat, 02 Apr 2022   Prob (F-statistic):               0.00
Time:                        20:42:37   Log-Likelihood:                -17750.
No. Observations:                1460   AIC:                         3.551e+04
Df Residuals:                    1456   BIC:                         3.553e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.213e+05   6618.751    -18.322   

# START OF HOMEWORK EXERCISE

## Data Exploration

Trying to understand the data

In [84]:
df=housing_data.copy()

In [85]:
column_names=df.columns.values
for column in column_names:
  print(column)
  print(df[column].value_counts(dropna=False))
  print(' ')
  print('------------------------------------------- ')

Id
1       1
982     1
980     1
979     1
978     1
       ..
485     1
484     1
483     1
482     1
1460    1
Name: Id, Length: 1460, dtype: int64
 
------------------------------------------- 
MSSubClass
20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64
 
------------------------------------------- 
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
 
------------------------------------------- 
LotFrontage
NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
65.0      44
85.0      40
78.0      25
90.0      23
21.0      23
64.0      19
24.0      19
68.0      19
73.0      18
55.0      17
72.0      17
63.0      17
79.0      17
100.0     16
66.0      15
51.0      15
74.0      15
52.0      14
59.0      13
82.0      12
71.0      12
67.0      12
57.0    

By looking at the column names, these are the variables that one can easily assume have a correlation with the housing market as they would likely steer buyers from or to a particular house:

In [86]:
var_to_study=[
 "LotArea",
 "YearBuilt",
 "HouseStyle",
 "OverallCond",
 "Street",
 "LandContour",
 "GrLivArea",
 "Utilities",
 "LotConfig",
 "LandSlope",
 "Neighborhood",
 "BldgType",
 "OverallQual",
 "Exterior1st",
 "ExterQual",
 "CentralAir",
 "1stFlrSF",
 "FullBath",
 "BedroomAbvGr",
 "KitchenQual",
 "TotRmsAbvGrd",
 "GarageType",
 "GarageCars",
 "PoolArea",
 "MoSold",
 "SalePrice"
]

In [87]:
#created ne df with the columns we want to study deeper
df2=df[var_to_study]
df2.head(10)

,LotArea,YearBuilt,HouseStyle,OverallCond,Street,LandContour,GrLivArea,Utilities,LotConfig,LandSlope,...,1stFlrSF,FullBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,GarageType,GarageCars,PoolArea,MoSold,SalePrice
0,8450,2003,2Story,5,Pave,Lvl,1710,AllPub,Inside,Gtl,...,856,2,3,Gd,8,Attchd,2,0,2,208500
1,9600,1976,1Story,8,Pave,Lvl,1262,AllPub,FR2,Gtl,...,1262,2,3,TA,6,Attchd,2,0,5,181500
2,11250,2001,2Story,5,Pave,Lvl,1786,AllPub,Inside,Gtl,...,920,2,3,Gd,6,Attchd,2,0,9,223500
3,9550,1915,2Story,5,Pave,Lvl,1717,AllPub,Corner,Gtl,...,961,1,3,Gd,7,Detchd,3,0,2,140000
4,14260,2000,2Story,5,Pave,Lvl,2198,AllPub,FR2,Gtl,...,1145,2,4,Gd,9,Attchd,3,0,12,250000
5,14115,1993,1.5Fin,5,Pave,Lvl,1362,AllPub,Inside,Gtl,...,796,1,1,TA,5,Attchd,2,0,10,143000
6,10084,2004,1Story,5,Pave,Lvl,1694,AllPub,Inside,Gtl,...,1694,2,3,Gd,7,Attchd,2,0,8,307000
7,10382,1973,2Story,6,Pave,Lvl,2090,AllPub,Corner,Gtl,...,1107,2,3,TA,7,Attchd,2,0,11,200000
8,6120,1931,1.5Fin,5,Pave,Lvl,1774,AllPub,Inside,Gtl,...,1022,2,2,TA,8,Detchd,2,0,4,129900
9,7420,1939,1.5Unf,6,Pave,Lvl,1077,AllPub,Corner,Gtl,...,1077,1,2,TA,5,Attchd,1,0,1,118000


In [88]:
# missing data
((df2
 .isna()
 .mean()
 *100)
 .pipe(lambda ser:ser[ser>0])
)

GarageType    5.547945
dtype: float64

In [89]:
#after seeing that only this column has missing data and that it is very difficult to handle, I decided to drop it
df3=df2.drop('GarageType', axis=1)
#we should also drop df5['Sales Price'] 
df3=df3.drop('SalePrice', axis=1)
df_check=df3.copy()

In [90]:
columns=[ 'Street','Utilities','CentralAir','LandSlope','KitchenQual','ExterQual','LandContour','LotConfig','BldgType','Neighborhood','Exterior1st' ]
for column in columns:
  print(column,sorted(df_check[column].unique()))



Street ['Grvl', 'Pave']
Utilities ['AllPub', 'NoSeWa']
CentralAir ['N', 'Y']
LandSlope ['Gtl', 'Mod', 'Sev']
KitchenQual ['Ex', 'Fa', 'Gd', 'TA']
ExterQual ['Ex', 'Fa', 'Gd', 'TA']
LandContour ['Bnk', 'HLS', 'Low', 'Lvl']
LotConfig ['Corner', 'CulDSac', 'FR2', 'FR3', 'Inside']
BldgType ['1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE']
Neighborhood ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker']
Exterior1st ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Plywood', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing']


## Preprocessing Data
Creating Dummies & Encoding categorical variables

In [91]:
df_ini=df_check.copy()
df_end=df_ini.copy()

In [92]:
#Encoding Central Air
column='CentralAir'
dic={'N':0,'Y':1}

new_column=column+'_enc'
df_result=df_end.copy()
df_result[new_column]=df_result[column].map(dic)
df_result=df_result.drop([column], axis=1)
print(df_result[new_column].value_counts())

df_end=df_result.copy()


1    1365
0      95
Name: CentralAir_enc, dtype: int64


In [93]:
#Encoding Street
column='Street'
dic={'Grvl':0,'Pave':1}

new_column=column+'_enc'
df_result=df_end.copy()
df_result[new_column]=df_result[column].map(dic)
df_result=df_result.drop([column], axis=1)
print(df_result[new_column].value_counts())

df_end=df_result.copy()

1    1454
0       6
Name: Street_enc, dtype: int64


In [94]:
#Encoding Utilities
column='Utilities'
dic={'NoSeWa':0,'AllPub':1}

new_column=column+'_enc'
df_result=df_end.copy()
df_result[new_column]=df_result[column].map(dic)
df_result=df_result.drop([column], axis=1)
print(df_result[new_column].value_counts())

df_end=df_result.copy()

1    1459
0       1
Name: Utilities_enc, dtype: int64


In [95]:
#Encoding Pool Area
column="PoolArea"
dic={0:0,480:1,512:1,519:1,555:1,576:1,648:1,738:1}

new_column=column+'_enc'
df_result=df_end.copy()
df_result[new_column]=df_result[column].map(dic)
df_result=df_result.drop([column], axis=1)
print(df_result[new_column].value_counts())

df_end=df_result.copy()

0    1453
1       7
Name: PoolArea_enc, dtype: int64


In [96]:
#Dummy Encode KitchenQual ['Ex', 'Fa', 'Gd', 'TA']
column='KitchenQual'

new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()


In [97]:
#Dummy Encode ExterQual ['Ex', 'Fa', 'Gd', 'TA']

#modify this:
column='ExterQual'


new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()



In [98]:
#Dummy Encode LotConfig ['Corner', 'CulDSac', 'FR2', 'FR3', 'Inside']

#modify this:
column='LotConfig'

new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()



In [99]:
#Dummy Encode LandContour ['Bnk', 'HLS', 'Low', 'Lvl']

#modify this:
column='LandContour'

new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()



In [100]:
column='LandSlope'

new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()



In [101]:
column='BldgType'

new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()

In [102]:
column='HouseStyle' 
new_column=column+'_dummies'
df_ini=df_end.copy()
df_result=df_end.copy()
new_names=[]
#create dummies
df_dummy=pd.get_dummies(df_ini[column])

#rename columns
column_names=df_ini[column].unique()
for name in column_names:
  new_column_name=column+'_'+name
  new_names.append(new_column_name)

df_dummy.columns=new_names
df_result=pd.concat([df_ini,df_dummy], axis=1)
df_result=df_result.drop(column,axis=1)
df_end=df_result.copy()

In [103]:
#dropping columns with too many values that can't be groupped
df_result=df_result.drop(['Neighborhood', 'Exterior1st'], axis=1)
df_end=df_result.copy()

In [104]:
df_result=df_end
print(df_result['OverallCond'])

0       5
1       8
2       5
3       5
4       5
       ..
1455    5
1456    6
1457    9
1458    6
1459    6
Name: OverallCond, Length: 1460, dtype: int64


In [114]:
columns=df_end.columns.values
for column in columns:
  print(column,sorted(df_result[column].unique()))


LotArea [1300, 1477, 1491, 1526, 1533, 1596, 1680, 1869, 1890, 1920, 1936, 1950, 1953, 1974, 2001, 2016, 2117, 2160, 2217, 2268, 2280, 2289, 2308, 2368, 2448, 2500, 2522, 2544, 2572, 2592, 2628, 2645, 2651, 2665, 2887, 3000, 3010, 3013, 3072, 3136, 3180, 3182, 3196, 3230, 3316, 3363, 3378, 3500, 3600, 3604, 3635, 3636, 3675, 3684, 3696, 3735, 3842, 3880, 3922, 3950, 3951, 3964, 3982, 4017, 4043, 4045, 4058, 4060, 4118, 4130, 4224, 4230, 4251, 4270, 4274, 4280, 4282, 4388, 4400, 4403, 4426, 4435, 4438, 4456, 4500, 4571, 4590, 4608, 4671, 4712, 4750, 4800, 4920, 4923, 4928, 5000, 5001, 5062, 5063, 5100, 5105, 5119, 5232, 5250, 5271, 5306, 5310, 5330, 5350, 5362, 5381, 5389, 5395, 5400, 5436, 5500, 5520, 5586, 5587, 5600, 5604, 5664, 5684, 5687, 5700, 5720, 5784, 5790, 5814, 5820, 5825, 5868, 5890, 5900, 5925, 6000, 6040, 6060, 6120, 6130, 6155, 6171, 6173, 6180, 6204, 6240, 6270, 6292, 6300, 6305, 6324, 6342, 6371, 6380, 6402, 6420, 6435, 6442, 6563, 6600, 6627, 6629, 6762, 6768, 6780, 6

## Evaluating Model and Improving

In [119]:
df_final=df_end.copy()

In [120]:
X=df_final.copy()
X = sm.add_constant(X)
X.head()
y = df2['SalePrice']
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     167.3
Date:                Sat, 02 Apr 2022   Prob (F-statistic):               0.00
Time:                        20:46:17   Log-Likelihood:                -17256.
No. Observations:                1460   AIC:                         3.460e+04
Df Residuals:                    1418   BIC:                         3.482e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              -3.73e+05   4.38e+0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Given the above information we can see that there are some variables that do not pass the T & P Test

Variables outside of 0.05<P:




## MY RESULTING MODEL

In [127]:
ini_names=df_end.columns.values
print(old_names)

['LotArea' 'YearBuilt' 'OverallCond' 'GrLivArea' 'OverallQual' '1stFlrSF'
 'FullBath' 'BedroomAbvGr' 'TotRmsAbvGrd' 'GarageCars' 'MoSold'
 'CentralAir_enc' 'Street_enc' 'Utilities_enc' 'PoolArea_enc'
 'KitchenQual_Gd' 'KitchenQual_TA' 'KitchenQual_Ex' 'KitchenQual_Fa'
 'ExterQual_Gd' 'ExterQual_TA' 'ExterQual_Ex' 'ExterQual_Fa'
 'LotConfig_Inside' 'LotConfig_FR2' 'LotConfig_Corner' 'LotConfig_CulDSac'
 'LotConfig_FR3' 'LandContour_Lvl' 'LandContour_Bnk' 'LandContour_Low'
 'LandContour_HLS' 'LandSlope_Gtl' 'LandSlope_Mod' 'LandSlope_Sev'
 'BldgType_1Fam' 'BldgType_2fmCon' 'BldgType_Duplex' 'BldgType_TwnhsE'
 'BldgType_Twnhs' 'HouseStyle_2Story' 'HouseStyle_1Story'
 'HouseStyle_1.5Fin' 'HouseStyle_1.5Unf' 'HouseStyle_SFoyer'
 'HouseStyle_SLvl' 'HouseStyle_2.5Unf' 'HouseStyle_2.5Fin']


In [128]:
#refitting model to have less variables
columns_dropped=['MoSold', 'ExterQual_Gd','ExterQual_TA' 'ExterQual_Ex' 'ExterQual_Fa', 'LandContour_Lvl', 'LandContour_Bnk', 'LandContour_Low',
       'LandContour_HLS','LandSlope_Mod' ]
columns=set(ini_names) - set(columns_dropped)
df_final=df_final[columns]
print(df_final.columns.values)

['OverallCond' 'ExterQual_Ex' 'KitchenQual_Ex' 'HouseStyle_2.5Unf'
 'LotConfig_Inside' 'LotConfig_FR2' 'Utilities_enc' 'LandSlope_Sev'
 'HouseStyle_SFoyer' 'LotArea' '1stFlrSF' 'CentralAir_enc'
 'HouseStyle_1Story' 'TotRmsAbvGrd' 'GrLivArea' 'KitchenQual_TA'
 'LotConfig_FR3' 'BldgType_2fmCon' 'BldgType_Twnhs' 'HouseStyle_SLvl'
 'BldgType_1Fam' 'HouseStyle_1.5Fin' 'GarageCars' 'ExterQual_Fa'
 'KitchenQual_Gd' 'LotConfig_Corner' 'HouseStyle_1.5Unf'
 'HouseStyle_2Story' 'OverallQual' 'HouseStyle_2.5Fin' 'LandSlope_Gtl'
 'Street_enc' 'KitchenQual_Fa' 'PoolArea_enc' 'BedroomAbvGr' 'FullBath'
 'BldgType_Duplex' 'YearBuilt' 'LotConfig_CulDSac' 'BldgType_TwnhsE'
 'ExterQual_TA']


In [126]:
X=df_final.copy()
X = sm.add_constant(X)
X.head()
y = df2['SalePrice']
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     182.0
Date:                Sat, 02 Apr 2022   Prob (F-statistic):               0.00
Time:                        20:50:49   Log-Likelihood:                -17269.
No. Observations:                1460   AIC:                         3.461e+04
Df Residuals:                    1422   BIC:                         3.481e+04
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              -5.39e+05   6.51e+0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
